#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categories(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 12, 15)

### Begin of Tables in the process

In [6]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,22,B,BH,500,200.00,0,5pct,SET50,2022-02-02


In [8]:
orders_log = pd.read_csv('../data/orders-log.csv')
orders_log['trans'] = orders_log.apply(lambda row: categories(row), axis=1)
orders_log['amount'] = orders_log.qty * orders_log.target
orders_log[orders_log.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
4,B,GVREIT,-2,C8.90,SET,10000,8.900000,9.000000,0.000000,0.00%,2,2022-12-08,Buy,89000.000000


In [9]:
orders_log.head().sort_values('spd',ascending=False).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
4,B,GVREIT,-2,C8.90,SET,"10,000",8.90,9.00,0.00,0.00%,2,2022-12-08,Buy,"89,000.00"
1,B,CKP,-3,3L,SET100,"20,000",4.60,4.66,-0.06,-1.27%,1,2022-02-02,Buy,"92,000.00"
3,B,GFPT,-3,XXX,SET,"7,500",12.50,12.80,-0.20,-1.54%,1,2022-02-02,Buy,"93,750.00"
0,B,BH,-10,5pct,SET50,500,200.00,210.00,-1.00,-0.47%,0,2022-02-02,Buy,"100,000.00"
2,B,CPF,-10,XXX,SET50,"4,000",23.00,24.00,0.20,+0.84%,0,2022-02-02,Buy,"92,000.00"


### End of Tables in the process

### Print to verify before upload file

In [10]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [11]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

In [12]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)

df_orders.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,xdate
0,B,GVREIT,"10,000",8.90,"89,000.00",C8.90,SET,2022-12-08


In [13]:
file_name = 'df_orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders.to_csv(output_file, header=True, index=False)
df_orders.to_csv(data_file, header=True, index=False)
df_orders.to_csv(box_file, header=True, index=False)
df_orders.to_csv(one_file, header=True, index=False)

### End of transactional process

In [14]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,GVREIT,"10,000",8.90,"89,000.00",C8.90,SET,2,2022-12-08


In [15]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    89000.0
Name: amount, dtype: float64

In [16]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,GVREIT,"10,000",8.90,"89,000.00",C8.90,SET,2,2022-12-08


In [17]:
cash = 240_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(240000, 89000.0, 151000.0)

In [18]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate


In [19]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(240000, 0.0, 240000.0)

### After call ord-log (must call everytime that orders change)

In [20]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categories(row), axis=1)
df['amount'] = df.qty * df.target

In [21]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [22]:
mb = (df.trans == 'Buy') & (df.spd >= -3)
df[mb].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,B,CKP,-3,3L,SET100,20000,4.6,4.66,-0.06,-1.27%,1,2022-02-02,Buy,92000.0
3,B,GFPT,-3,XXX,SET,7500,12.5,12.80,-0.20,-1.54%,1,2022-02-02,Buy,93750.0
4,B,GVREIT,-2,C8.90,SET,10000,8.9,9.00,0.00,0.00%,2,2022-12-08,Buy,89000.0
9,B,SCB,-3,XXX,SET50,1000,102.0,103.50,0.00,0.00%,1,2022-02-14,Buy,102000.0


In [23]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [24]:
ms = (df.trans == 'Sell') & (df.spd <= 3)
df[ms].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount


In [25]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,B,CKP,-3,3L,SET100,20000,4.6,4.66,-0.06,-1.27%,1,2022-02-02,Buy,92000.0
3,B,GFPT,-3,XXX,SET,7500,12.5,12.80,-0.20,-1.54%,1,2022-02-02,Buy,93750.0
4,B,GVREIT,-2,C8.90,SET,10000,8.9,9.00,0.00,0.00%,2,2022-12-08,Buy,89000.0
9,B,SCB,-3,XXX,SET50,1000,102.0,103.50,0.00,0.00%,1,2022-02-14,Buy,102000.0


In [26]:
df[mb | ms].shape[0]

4

In [27]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
7,PTTGC,Buy,"123,750.00",-24
6,JMART,Buy,"120,000.00",-4
9,SCB,Buy,"102,000.00",-3
0,BH,Buy,"100,000.00",-10
5,III,Buy,"100,000.00",-11


### Select source of orders between these two

In [28]:
df_out = df[mb | ms][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
1,Buy,CKP,20000,4.6,1,-3,4.66,-0.06,-1.27%,3L,SET100,2022-02-02
3,Buy,GFPT,7500,12.5,1,-3,12.80,-0.20,-1.54%,XXX,SET,2022-02-02
4,Buy,GVREIT,10000,8.9,2,-2,9.00,0.00,0.00%,C8.90,SET,2022-12-08
9,Buy,SCB,1000,102.0,1,-3,103.50,0.00,0.00%,XXX,SET50,2022-02-14


In [29]:
df_out = df[df.active == 2][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
4,Buy,GVREIT,10000,8.9,2,-2,9.0,0.0,0.00%,C8.90,SET,2022-12-08


In [30]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [31]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,GVREIT,"10,000",8.90,"89,000.00",C8.90,SET,2,2022-12-08


### Evening Process

In [32]:
new_active = 1
old_active = 2

In [33]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

1

In [34]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'GVREIT'"

In [35]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

1

In [36]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process